In [ ]:
pip uninstall -y torch torchvision torch2trt tensorrt


Found existing installation: tensorrt 10.9.0.34
Uninstalling tensorrt-10.9.0.34:
  Successfully uninstalled tensorrt-10.9.0.34


In [ ]:
pip install torch==2.1.2+cu118 torchvision==0.16.2+cu118 -f https://download.pytorch.org/whl/cu118/torch_stable.html


Looking in links: https://download.pytorch.org/whl/cu118/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 531.7 kB/s eta 0:00:00
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.16.2%2Bcu118-cp311-cp311-linux_x86_64.whl (6.1 MB)
  Using cached triton-2.1.0-0-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 8.7 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.1.0
    Uninstalling triton-3.1.0:
      Successfully uninstalled triton-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.1+cu124 requires torch==2.5.1, but you have torch 2.1.2+cu118 which is incompatible.


Install PyTorch + torchvision (Matching CUDA 11.8)

In [ ]:
!python3 -m pip install --upgrade tensorrt

  Using cached tensorrt-10.9.0.34-py2.py3-none-any.whl


In [ ]:
import torch
import tensorrt as trt

print("✅ CUDA Available:", torch.cuda.is_available())
print("✅ CUDA Version:", torch.version.cuda)
print("✅ TensorRT Version:", trt.__version__)


✅ CUDA Available: True
✅ CUDA Version: 11.8
✅ TensorRT Version: 10.9.0.34


In [ ]:
!pip install torch2trt

ERROR: Could not find a version that satisfies the requirement torch2trt (from versions: none)
ERROR: No matching distribution found for torch2trt


In [ ]:
!pip install nvidia-pyindex
!pip install nvidia-tensorrt
!pip install git+https://github.com/NVIDIA-AI-IOT/torch2trt.git


  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-pyindex: filename=nvidia_pyindex-1.0.9-py3-none-any.whl size=8419 sha256=b172a068f050ec1a68e567c7ba70e617476840a4b8d8a38cb495f5321d1f19ab
  Stored in directory: /root/.cache/pip/wheels/49/d0/7d/b68b3665d16ee20355e65fb7ef48b7ca26533217d9f09924fe
Successfully built nvidia-pyindex
  Cloning https://github.com/NVIDIA-AI-IOT/torch2trt.git to /tmp/pip-req-build-8i8waybd
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA-AI-IOT/torch2trt.git /tmp/pip-req-build-8i8waybd
  Resolved https://github.com/NVIDIA-AI-IOT/torch2trt.git to commit 4e820ae31b4e35d59685935223b05b2e11d47b03
  Preparing metadata (setup.py) ... done
  Created wheel for torch2trt: filename=torch2trt-0.5.0-py3-none-any.whl size=53585 sha256=4a6c01a52952ea008187d2a9190fd860fa734686ba284966a8d1812a4685af14
  Stored in directory: /tmp/pip-ephem-wheel-cache-ccapv_9h/wheels/9f/c1/98/8424c6521a320e9939b2c3d61bf590819712872ad035555def
Suc

In [ ]:
from torch2trt import torch2trt
import torch
import torchvision.models as models

# Load a pre-trained ResNet50
model = models.resnet50(pretrained=True).cuda()

# Convert model to TensorRT
x = torch.randn(1, 3, 224, 224).cuda()
model_trt = torch2trt(model, [x])

# Test inference
out = model_trt(x)
print("✅ Successfully converted to TensorRT")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 75.1MB/s]


✅ Successfully converted to TensorRT


In [ ]:
print(type(model_trt))


<class 'torch2trt.trt_module.TRTModule'>


Baseline pytorch inference


In [ ]:
import torch
import torchvision.models as models
import time


model = models.resnet50(pretrained=True).cuda()
model.eval()

# a dummy inp
x = torch.randn(1, 3, 224, 224).cuda()

# measuring the inf time
start_time = time.time()
with torch.no_grad():
    output = model(x)
end_time = time.time()

# Calc time
pytorch_time = (end_time - start_time) * 1000
print(f" PyTorch Inference Time: {pytorch_time:.2f} ms")


 PyTorch Inference Time: 7.25 ms


Tensor RT Inference

In [ ]:
from torch2trt import torch2trt
import time

#convert pytorch to tensorRT
model = models.resnet50(pretrained=True).cuda()
x = torch.randn(1, 3, 224, 224).cuda()
model_trt = torch2trt(model, [x])

# measure inf time
start_time = time.time()
with torch.no_grad():
    output_trt = model_trt(x)
end_time = time.time()

# Calc tme
tensorrt_time = (end_time - start_time) * 1000
print(f" TensorRT Inference Time: {tensorrt_time:.2f} ms")


 TensorRT Inference Time: 1.72 ms


## Whisper Medium





In [ ]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-59d798kt
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-59d798kt
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.8 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803670 sha256=16850f6fe226d953fa69eda161d3d0e788c9e74c719c0d16f7539854b7ab252d
  Stored in directory: /tmp/pip-ephem-wheel-cache-x35c9nfl/wheels/1f/1d/98/9583695e6695a6ac0ad42d87511097dce5ba486647dbfecb0e
Successfully built openai-whisper


In [ ]:
import torch
import whisper

model = whisper.load_model("medium")
model.eval().cuda()


100%|█████████████████████████████████████| 1.42G/1.42G [00:53<00:00, 28.4MiB/s]


Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(1024, 1024, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-23): 24 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=1024, out_features=1024, bias=True)
          (key): Linear(in_features=1024, out_features=1024, bias=False)
          (value): Linear(in_features=1024, out_features=1024, bias=True)
          (out): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (attn_ln): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=1024, out_features=4096, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=4096, out_features=1024, bias=True)
        )
        (mlp_ln): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm((

In [ ]:
from transformers import WhisperForConditionalGeneration
import torch

# Load Whisper-medium from Hugging Face
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium").cuda()

# Set to evaluation mode
model.eval()


RuntimeError: Failed to import transformers.models.whisper.modeling_whisper because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
cannot import name '_resolve_process_group' from 'torch._C._distributed_c10d' (unknown location)